<a href="https://colab.research.google.com/github/Anavi07/AudioSpotting/blob/main/Bangla_AlexNet_MS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
# Paths
main_dir = '/content/Bangla-Number-Recognition-CNN'  # Path to FSDD
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

if not os.path.exists(files_path):
    os.makedirs(files_path)

# Clone the dataset if not already cloned
if not os.path.exists(main_dir):
    !git clone https://github.com/ClockWorkKid/Bangla-Number-Recognition-CNN


Cloning into 'Bangla-Number-Recognition-CNN'...
remote: Enumerating objects: 2699, done.
remote: Total 2699 (delta 0), reused 0 (delta 0), pack-reused 2699 (from 1)
Receiving objects: 100% (2699/2699), 85.13 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Updating files: 100% (2657/2657), done.


In [4]:
# PROCESSING
# Feature extraction methods

# def CalculateZeroCrossingRate(file_location):
#     y, sr = librosa.load(file_location)
#     zCross = librosa.feature.zero_crossing_rate(y=y)
#     dim = (32, 32)
#     resized = cv2.resize(zCross, dim, interpolation = cv2.INTER_AREA)
#     return resized

def CalculateMelSpectrogram(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
    melSpec_dB = librosa.power_to_db(melSpec)
    dim = (32, 32)
    resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
    return resized


def CalculateMFCC(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.mfcc(y=y, sr=sr)
    dim = (32, 32)
    resized = cv2.resize(melSpec, dim, interpolation = cv2.INTER_AREA)
    return resized



In [5]:
cd /content

/content


In [ ]:
'''for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break
  break '''

'for i in range(10):\n  j = \'/content/dataset\'\n  print(os.listdir(j))\n  for k in os.listdir(j):\n    print(k)\n    if ".DS_" in k:\n      continue\n\n    for t in os.listdir(j+"/"+k):\n      #print("path = ", os.listdir(j+"/"+k))\n      if ".DS_" in k:\n        continue\n      print(t.split("WA")[1].split(".")[0])\n      #print(t.split("D")[1][0])\n      break\n    break\n  break '

In [6]:
import os
import numpy as np

# Paths
main_dir = '/content/Bangla-Number-Recognition-CNN'  # Path to the main dataset folder
recordings_dir = os.path.join(main_dir, 'Dataset/Dataset_sorted')  # Path to the Dataset_sorted folder
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

# Ensure output directory exists
if not os.path.exists(files_path):
    os.makedirs(files_path)

# Processing
x = []
y = []

# Loop through each label directory (0, 1, ..., 9, noise)
for label_dir in os.listdir(recordings_dir):
    label_path = os.path.join(recordings_dir, label_dir)

    # Ensure it's a directory
    if not os.path.isdir(label_path):
        continue

    # Process files within the label directory
    for file_name in os.listdir(label_path):
        if not file_name.endswith('.wav'):  # Skip non-wav files
            continue

        file_path = os.path.join(label_path, file_name)

        try:
            # Extract Mel Spectrogram features
            x.append(CalculateMelSpectrogram(file_path))

            # Use directory name as label (e.g., '0', '1', ..., 'noise')
            label = int(label_dir) if label_dir.isdigit() else 10  # Assign 10 for 'noise'
            y.append(label)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Convert to NumPy arrays
x = np.array(x)
y = np.array(y)

# Verify data
if x.size == 0 or y.size == 0:
    raise ValueError("No data was processed. Check dataset structure and feature extraction.")

# Save processed data
np.save(os.path.join(files_path, "X_melspectrogram.npy"), x)
np.save(os.path.join(files_path, "Y_melspectrogram.npy"), y)

print(f"Processing completed. Processed data saved in {files_path}.")


Processing completed. Processed data saved in /content/drive/MyDrive/bangla.


In [7]:
#load files
files_path = '/content/drive/MyDrive/bangla'  # Save processed data

x_melspectrogram = np.load(files_path+"/X_melspectrogram.npy", allow_pickle=True)
# x_zero_crossing_rate = np.load(files_path+"/X_zero_crossing_rate.npy", allow_pickle=True)
#x_mfcc = np.load(files_path+"/X_mfcc.npy", allow_pickle=True)
y_melspectrogram = np.load(files_path+"/Y_melspectrogram.npy", allow_pickle=True)
# y_zero_crossing_rate = np.load(files_path+"/y_zero_crossing_rate.npy", allow_pickle=True)
#y_mfcc = np.load(files_path+"/Y_mfcc.npy", allow_pickle=True)

In [ ]:
# #Split data into train and test

# from sklearn.model_selection import train_test_split
# x = x_mfcc
# y = y_mfcc

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_mfcc_train",x_train)
# np.save(files_path+"/y_mfcc_train",y_train)
# np.save(files_path+"/x_mfcc_test",x_test)
# np.save(files_path+"/y_mfcc_test",y_test)

In [8]:
from sklearn.model_selection import train_test_split
x = x_melspectrogram
y = y_melspectrogram

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_melspectrogram_train",x_train)
np.save(files_path+"/y_melspectrogram_train",y_train)
np.save(files_path+"/x_melspectrogram_test",x_test)
np.save(files_path+"/y_melspectrogram_test",y_test)

In [ ]:

# from sklearn.model_selection import train_test_split
# x = x_zero_crossing_rate
# y = y_zero_crossing_rate

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_zero_crossing_rate_train",x_train)
# np.save(files_path+"/y_zero_crossing_rate_train",y_train)
# np.save(files_path+"/x_zero_crossing_rate_test",x_test)
# np.save(files_path+"/y_zero_crossing_rate_test",y_test)

In [9]:
import tensorflow as tf
import numpy as np

files_path = '/content/drive/MyDrive/bangla'

features = ['melspectrogram', 'mfcc', 'zero_crossing_rate']

x_train = np.load(files_path+"/x_{}_train.npy".format(features[0]), allow_pickle=True)
y_train = np.load(files_path+"/y_{}_train.npy".format(features[0]), allow_pickle=True)

x_test = np.load(files_path+"/x_{}_test.npy".format(features[0]),allow_pickle=True)
y_test = np.load(files_path+"/y_{}_test.npy".format(features[0]),allow_pickle=True)

In [10]:
print(np.unique(y_melspectrogram))

[ 0  1  2  3  4  5  6  7  8  9 10]


In [11]:
x_train =x_train.reshape((x_train.shape[0],32,32,1))
x_test =x_test.reshape((x_test.shape[0],32,32,1))

In [12]:
x_train = x_train.reshape(-1, 32, 32, 1)  # Reshape to match the input shape of the model
x_test = x_test.reshape(-1, 32, 32, 1)    # Reshape for testing
train_y = tf.keras.utils.to_categorical(y_train, num_classes=11)  # One-hot encode
test_y = tf.keras.utils.to_categorical(y_test, num_classes=11)    # One-hot encode


## Train Model

In [14]:
#AlexNet
import tensorflow as tf
from tensorflow.keras import layers, models

# Define AlexNet model
def GetAlexNet():
    model = models.Sequential()

    # Layer 1: Convolutional + MaxPooling
    model.add(layers.Conv2D(96, (3, 3), strides=(1, 1), activation='relu', input_shape=(32, 32, 1), padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # Layer 2: Convolutional + MaxPooling
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # Layer 3: Convolutional
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # Layer 4: Convolutional
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # Layer 5: Convolutional + MaxPooling
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # Flatten the output
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(11, activation='softmax'))  # For 11 classes

    return model

# Preprocess data
x_train = x_train.reshape(-1, 32, 32, 1)  # Reshape to match input shape
x_test = x_test.reshape(-1, 32, 32, 1)    # Reshape for testing
train_y = tf.keras.utils.to_categorical(y_train, num_classes=11)  # One-hot encode labels
test_y = tf.keras.utils.to_categorical(y_test, num_classes=11)    # One-hot encode labels

# Define model and compile
alexnet_model = GetAlexNet()
alexnet_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

# Define callback for saving the best model
callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/auddsavedfiles/models/alexnet_model_.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Train the model
history = alexnet_model.fit(
    x_train, train_y,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1,
    callbacks=[callbacks]
)




Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.0806 - loss: 176203.1406
Epoch 1: val_accuracy improved from -inf to 0.10152, saving model to /content/drive/MyDrive/auddsavedfiles/models/alexnet_model_.keras
28/28 ━━━━━━━━━━━━━━━━━━━━ 20s 395ms/step - accuracy: 0.0807 - loss: 172238.2812 - val_accuracy: 0.1015 - val_loss: 21.3831
Epoch 2/50
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1398 - loss: 2.6429
Epoch 2: val_accuracy improved from 0.10152 to 0.11168, saving model to /content/drive/MyDrive/auddsavedfiles/models/alexnet_model_.keras
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 163ms/step - accuracy: 0.1385 - loss: 2.6307 - val_accuracy: 0.1117 - val_loss: 2.3843
Epoch 3/50
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1060 - loss: 2.4975
Epoch 3: val_accuracy did not improve from 0.11168
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.1061 - loss: 2.4929 - val_accuracy: 0.1066 - val_loss: 2.3940
Epoch 4/50
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accur

In [15]:
# Evaluate the model
alexnet_model = tf.keras.models.load_model('/content/drive/MyDrive/auddsavedfiles/models/alexnet_model_.keras')
print("Test Accuracy is ", alexnet_model.evaluate(x_test, test_y)[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.1024 - loss: 2.3840
Test Accuracy is  0.12219959497451782
